In [6]:
import numpy as np
import pandas as pd 

from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
import catboost
import optuna

RANDOM_SEED = 121

Что попробовать: 
- эмбеддинги
- optuna 
- новые фичи 
- убрать стоп слова 

In [2]:
df = pd.read_csv('../data/interim/df_text.csv')
df = df[['text', 'level']]

In [3]:
train_df, test_df = train_test_split(df, train_size=0.8, random_state=0)
y_train, X_train = train_df['level'], train_df.drop(['level'], axis=1)
y_test, X_test = test_df['level'], test_df.drop(['level'], axis=1)

train_pool = Pool(data=X_train, label=y_train, text_features=['text'])
test_pool = Pool(data=X_test, label=y_test, text_features=['text'])

In [4]:
def fit_model_cv(train_pool):
    model = cv(
        pool=train_pool,
        params={'iterations':10000,
        'learning_rate':0.05, 
        'eval_metric':'TotalF1',
        'task_type':'GPU',
        'loss_function':'MultiClass', 
        'devices':[0, 1], 
        'auto_class_weights':'Balanced', 
        'l2_leaf_reg':11, 
        'random_seed':1        
        },
        return_models=True,
        stratified=True, 
        early_stopping_rounds=200,
        fold_count=5, 
        verbose=200

    )
    print("Лучшая метрика:", model[0]['test-TotalF1-mean'].max())
    print("Лучшая итерация:", np.argmax(model[0]['test-TotalF1-mean']))


fit_model_cv(train_pool)

Training on fold [0/5]
0:	learn: 0.5099637	test: 0.4585734	best: 0.4585734 (0)	total: 16.7ms	remaining: 2m 46s
200:	learn: 0.8899296	test: 0.5686095	best: 0.5697143 (194)	total: 2.26s	remaining: 1m 50s
400:	learn: 0.9616037	test: 0.5501817	best: 0.5724553 (286)	total: 4.48s	remaining: 1m 47s
bestTest = 0.5724553017
bestIteration = 286
Training on fold [1/5]
0:	learn: 0.5453920	test: 0.5439223	best: 0.5439223 (0)	total: 16.1ms	remaining: 2m 40s
200:	learn: 0.8862911	test: 0.5654005	best: 0.5854475 (168)	total: 2.23s	remaining: 1m 48s
bestTest = 0.5854474593
bestIteration = 168
Training on fold [2/5]
0:	learn: 0.5190278	test: 0.4806253	best: 0.4806253 (0)	total: 15.3ms	remaining: 2m 32s
200:	learn: 0.8871758	test: 0.5308642	best: 0.5438507 (175)	total: 2.22s	remaining: 1m 48s
bestTest = 0.5438507278
bestIteration = 175
Training on fold [3/5]
0:	learn: 0.5297759	test: 0.5171473	best: 0.5171473 (0)	total: 16.6ms	remaining: 2m 46s
200:	learn: 0.8898671	test: 0.5434841	best: 0.5767220 (83)	t

10 
Лучшая метрика: 0.5525073863678922
Лучшая итерация: 168



https://www.youtube.com/watch?v=lB0WYoz5nU4

In [5]:
# OPTUNA

In [19]:
best_iter, best_score = None, float('-inf')
def objective(trial):
    global best_iter, best_score

    param = {'iterations':10000,
             'early_stopping_rounds':300,
             'task_type':'GPU', 
             'devices':'0',
             'eval_metric':'TotalF1',
             'loss_function':'MultiClass', 
             'random_seed':RANDOM_SEED,
             'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.01), 
             "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 50), 
             #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.8), 
             "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
             "depth": trial.suggest_int("depth", 3, 9),
             "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"]),
             'eval_metric':'TotalF1', 
             }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 20)
        
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
    
    stat = cv(
        pool=train_pool,
        params=param,
        stratified=True,
        early_stopping_rounds=300,
        fold_count=5, 
        logging_level="Silent"
    )
    score = stat['test-TotalF1-mean'].max()
    iterations = np.argmax(stat['test-TotalF1-mean'])
    if score > best_score:
        best_score = score
        best_iter = iterations
    return score
    

In [20]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-06-23 19:02:28,603] A new study created in memory with name: no-name-6b53c520-6bac-4c4c-af49-c613613d1707


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-06-23 19:07:37,917] Trial 0 finished with value: 0.5064561450829568 and parameters: {'learning_rate': 0.0072797364586489844, 'l2_leaf_reg': 25, 'auto_class_weights': 'Balanced', 'depth': 9, 'bootstrap_type': 'Bernoulli', 'subsample': 0.6266089240922291}. Best is trial 0 with value: 0.5064561450829568.
[I 2023-06-23 19:12:29,370] Trial 1 finished with value: 0.5325872152131108 and parameters: {'learning_rate': 0.008163246587783984, 'l2_leaf_reg': 5, 'auto_class_weights': 'None', 'depth': 4, 'bootstrap_type': 'Bernoulli', 'subsample': 0.1645913082321564}. Best is trial 1 with value: 0.5325872152131108.
[I 2023-06-23 19:17:31,155] Trial 2 finished with value: 0.5106902453569023 and parameters: {'learning_rate': 0.007697202710883541, 'l2_leaf_reg': 31, 'auto_class_weights': 'SqrtBalanced', 'depth': 8, 'bootstrap_type': 'Bernoulli', 'subsample': 0.18131701995921673}. Best is trial 1 with value: 0.5325872152131108.
[I 2023-06-23 19:22:51,606] Trial 3 finished with value: 0.5279783387

: 

: 